In [1]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob

## 데이터 로드

In [2]:
ohlcv_df_raw = pd.read_csv('ohlcv_df_raw.csv', dtype = {'ticker':str})
ticker_list = ohlcv_df_raw['ticker'].unique()

In [19]:
diff_date = 60
now = datetime.now()
now = now + timedelta(days=-diff_date)
set_date = now.strftime('%Y-%m-%d')
df1 =  ohlcv_df_raw[ohlcv_df_raw['date'] > set_date].groupby("ticker")['date'].min().reset_index()


,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market
0,2021-01-04,81000,84400,80200,83000,38655276,2.469136,005930,삼성전자,KOSPI
48873,2021-01-04,37021,37362,36535,37216,1485830,2.135134,000720,현대건설,KOSPI
48176,2021-01-04,9450,9520,9250,9500,2346432,0.635593,006800,미래에셋증권,KOSPI
47479,2021-01-04,29500,29550,29150,29400,284656,-0.843170,241560,두산밥캣,KOSPI
46782,2021-01-04,382500,385000,378000,384500,94480,0.918635,097950,CJ제일제당,KOSPI
...,...,...,...,...,...,...,...,...,...,...
26589,2023-10-30,32250,32600,32150,32400,320466,0.000000,030200,KT,KOSPI
25892,2023-10-30,18550,18840,18200,18460,1623322,-1.703940,323410,카카오뱅크,KOSPI
25344,2023-10-30,13450,13790,13450,13780,1950548,1.472754,034020,두산에너빌리티,KOSPI
24647,2023-10-30,11420,11490,10970,11140,2440552,-3.298611,024110,기업은행,KOSPI


### 보조지표 계산

In [4]:
df_raw_total = pd.DataFrame()
df_raw_anal_total = pd.DataFrame()

for ticker_nm in ticker_list:
    df_raw = ohlcv_df_raw[ohlcv_df_raw['ticker'] == ticker_nm].reset_index(drop = True)
    
    ######################################################################
    # 보조지표
    ###################################################################### 

    # 이동평균선
    df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
    df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
    df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
    df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

    # 볼린저밴드
    std = df_raw['close'].rolling(20).std(ddof=0)

    df_raw['upper'] = df_raw['MA20'] + 2 * std
    df_raw['lower'] = df_raw['MA20'] - 2 * std

    # MACD
    # MACD 
    macd = MACD(close=df_raw['close'], 
                window_slow=26,
                window_fast=12, 
                window_sign=9)


    df_raw['MACD_DIFF'] = macd.macd_diff()
    df_raw['MACD'] = macd.macd()
    df_raw['MACD_Signal'] = macd.macd_signal()

    # RSI
    df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
    df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
    df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

    # welles moving average
    df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

    df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
    df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
    df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']
    
    
    ######################################################################
    # 보조지표 분석
    ###################################################################### 
    df_raw_anal = df_raw[['date','ticker', 'close']]
    
    # 골든크로스
    # 골든 크로스 5-20
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

    # 데드 크로스 5-20
    # 양수에서 음수로 바뀌는 모든 인덱스 찾기
    idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]
    

    df_raw_anal.loc[:, '5_20_cross'] = '-'
    df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

    df_raw_anal.loc[:, '20_60_cross'] = '-'
    df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'
    
    
    # 정배열 역배열
    ascending_sq  = (df_raw['MA5-20'] > 0) & \
    (df_raw['MA20-60'] > 0) & \
    (df_raw['MA60-120'] > 0) 

    descending_sq  = (df_raw['MA5-20'] < 0) & \
    (df_raw['MA20-60'] < 0) & \
    (df_raw['MA60-120'] < 0) 
    
    df_raw_anal.loc[:,'array'] = '-'
    df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
    df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'
    
    
    # 볼린저밴드
    down_reg_sq = df_raw['upper'] - df_raw['close'] 
    top_reg_sq  = df_raw['lower'] - df_raw['close'] 

    down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
    top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]
    
    df_raw_anal.loc[:,'Bollinger_band'] = '-'
    df_raw_anal.loc[ascending_sq,'Bollinger_band'] = '하향회귀(매도)'
    df_raw_anal.loc[descending_sq,'Bollinger_band'] = '상향회귀(매수)'
    
    
    # MACD
    signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
    signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]

    df_raw_anal.loc[:,'MACD'] = '-'
    df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
    df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'    
    
    # RSI
    down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
    top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]
    
    
    df_raw_anal.loc[:,'RSI'] = '-'
    df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
    df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'


    df_raw_total = pd.concat([df_raw_total, df_raw])
    df_raw_anal_total = pd.concat([df_raw_anal_total, df_raw_anal])
    
    print(ticker_nm)

/tmp/ipykernel_110462/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

005930
373220
000660


/tmp/ipykernel_110462/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

207940
005380
005490


/tmp/ipykernel_110462/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

...

/tmp/ipykernel_110462/984985259.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'Bollinger_band'] = '-'
/tmp/ipykernel_110462/984985259.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'MACD'] = '-'
/tmp/ipykernel_110462/984985259.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

036460
008930
002790


/tmp/ipykernel_110462/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

139480
012750
023530
137310


/tmp/ipykernel_110462/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_110462/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [5]:
df_raw_total = df_raw_total.reset_index(drop = True)

In [20]:
df_raw_anal_total

,date,ticker,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2021-01-04,005930,83000,-,-,-,-,-,-
1,2021-01-05,005930,83900,-,-,-,-,-,-
2,2021-01-06,005930,82200,-,-,-,-,-,-
3,2021-01-07,005930,82900,-,-,-,-,-,-
4,2021-01-08,005930,88800,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...
558,2023-10-24,137310,10280,-,-,역배열(매도),상향회귀(매수),상향돌파(매수),-
559,2023-10-25,137310,10310,-,-,역배열(매도),상향회귀(매수),-,-
560,2023-10-26,137310,9820,-,-,역배열(매도),상향회귀(매수),-,-
561,2023-10-27,137310,9690,-,-,역배열(매도),상향회귀(매수),하향돌파(매도),-


In [18]:
df_raw_total.columns

df_raw_total[['date', 'open', 'high', 'low', 'close', 'volume', 'price_change_percentage', 'ticker', 'corp_name', 'market', 'MA5',
       'MA20', 'MA60', 'MA120', 'upper', 'lower']]

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,MA5,MA20,MA60,MA120,upper,lower
0,2021-01-04,81000,84400,80200,83000,38655276,2.469136,005930,삼성전자,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-05,81600,83900,81600,83900,35335669,1.084337,005930,삼성전자,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-06,83300,84500,82100,82200,42089013,-2.026222,005930,삼성전자,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-07,82800,84200,82700,82900,32644642,0.851582,005930,삼성전자,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-08,83300,90000,83000,88800,59013307,7.117008,005930,삼성전자,KOSPI,84160.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68156,2023-10-24,10170,10300,9870,10280,220969,1.280788,137310,에스디바이오센서,KOSPI,10268.0,10660.0,11828.500000,13734.166667,11737.070100,9582.929900
68157,2023-10-25,10270,10420,10100,10310,202518,0.291829,137310,에스디바이오센서,KOSPI,10230.0,10583.0,11808.000000,13636.750000,11519.826558,9646.173442
68158,2023-10-26,10100,10130,9800,9820,406661,-4.752667,137310,에스디바이오센서,KOSPI,10116.0,10496.5,11786.833333,13546.500000,11378.050339,9614.949661
68159,2023-10-27,9900,9970,9640,9690,283074,-1.323829,137310,에스디바이오센서,KOSPI,10050.0,10411.5,11741.166667,13453.916667,11259.232859,9563.767141


### 등락률 계산

In [35]:
diff_date = 60
now = datetime.now()
now = now + timedelta(days=-diff_date)
set_date = now.strftime('%Y-%m-%d')
df1 =  ohlcv_df_raw[ohlcv_df_raw['date'] > set_date].groupby("ticker")['date'].min().reset_index()


In [36]:
diff_date_list = [30, 90, 180, 240, 365]
now = datetime.now()
set_date = now.strftime('%Y-%m-%d')

ohlcv_df_raw['date'] = pd.to_datetime(ohlcv_df_raw['date'])


df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] > '2023-10-01'].groupby("ticker")['date'].max().reset_index()
# df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] == '2023-10-06'].groupby("ticker")['date'].max().reset_index()
df1_set = df1_set[['ticker', 'date']]
df1_set = pd.merge(df1_set, ohlcv_df_raw[['ticker', 'date', 'close', 'corp_name']],
              on = ['ticker', 'date'], 
              how = 'left')

    
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  ohlcv_df_raw[ohlcv_df_raw['date'] > set_date].groupby("ticker")['date'].min().reset_index()

    
    df1 = df1[['ticker','date']]
    df1 = pd.merge(df1, ohlcv_df_raw[['ticker', 'date', 'close']],
                  on = ['ticker', 'date'], 
                  how = 'left')
    
    df1.columns = ['ticker', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'ticker', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}']    

In [37]:
df1_set_2 = df1_set[['date','ticker','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'corp_name']]

In [38]:
df_per_total = pd.DataFrame()
per_set = ['per_30', 'per_90', 'per_180', 'per_240', 'per_365']
ascending_list = [True, False]

for per_value in per_set:
    for ascending_value in ascending_list:
        df_per =  df1_set_2.sort_values(by =per_value, ascending = ascending_value).head()
        df_per['type'] = per_value
        df_per['rank'] = range(1,6)
        df_per_total = pd.concat([df_per_total, df_per])

df_per_total = df_per_total.drop_duplicates()        

## 최종

In [39]:
df_raw_total.head()

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
0,2021-01-04,81000,84400,80200,83000,38655276,2.469136,005930,삼성전자,KOSPI,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-05,81600,83900,81600,83900,35335669,1.084337,005930,삼성전자,KOSPI,...,NaN,900.0,900.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-06,83300,84500,82100,82200,42089013,-2.026222,005930,삼성전자,KOSPI,...,NaN,-1700.0,0.0,1700.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-07,82800,84200,82700,82900,32644642,0.851582,005930,삼성전자,KOSPI,...,NaN,700.0,700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-08,83300,90000,83000,88800,59013307,7.117008,005930,삼성전자,KOSPI,...,NaN,5900.0,5900.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_raw_anal_total.head()

,date,ticker,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2021-01-04,005930,83000,-,-,-,-,-,-
1,2021-01-05,005930,83900,-,-,-,-,-,-
2,2021-01-06,005930,82200,-,-,-,-,-,-
3,2021-01-07,005930,82900,-,-,-,-,-,-
4,2021-01-08,005930,88800,-,-,-,-,-,-


In [41]:
df_per_total.head()

,date,ticker,per_30,per_90,per_180,per_240,per_365,corp_name,type,rank
63,2023-10-30,051900,-0.289954,-0.294785,-0.470187,-0.536513,-0.408745,LG생활건강,per_30,1
0,2023-10-30,000100,-0.277215,-0.223129,-0.008681,0.077358,-0.011735,유한양행,per_30,2
40,2023-10-30,018880,-0.246285,-0.166667,-0.251844,-0.255765,-0.095541,한온시스템,per_30,3
9,2023-10-30,003670,-0.239583,-0.476970,-0.205288,0.089552,0.252451,포스코퓨처엠,per_30,4
20,2023-10-30,008770,-0.231353,-0.172789,-0.263030,-0.224490,-0.068913,호텔신라,per_30,5
